<a href="https://colab.research.google.com/github/PrajnyaKulkarni/aws/blob/main/Faker_with_aws_postgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generating fake data using the python Faker**
Documentation - https://faker.readthedocs.io/en/master/

## **Use Case**
In the project I was working on, I utilised this process to evaluate the data analysis module that was used to get the insights. We needed to test the work before releasing it to production, so we used the Faker class to produce the data. Every time we finished testing, we destroyed all of the objects we had built, including the database. AWS RDS was used.

## In this notebook, we will

1.   Create a database in Postgres, which is already provisioned in AWS
2.   Create Table
3.   Generate the Data
4.   Perform the exploratory data analysis

Here I will explain how to use the Faker class and I assume you know python and and have access to the Postgres database. Other databases can also be used.

I have used the following packages in the project:

If you do not have these packages the use pip install to get the packages

1.   **psycopg2**  is a PostgreSQL database driver,it is used to perform operations on PostgreSQL using python.
2.  **pandas** is a Python library used for working with datasets.
3. **random** is an in-built module of Python that is used to generate random numbers in Python.
4. **sql_queries** is module is created to drop and create tables. It is attached with this file.
5. **faker** is a python library to generate fake data
6. **config parser** is used to store the credentials for testing.  
documentation - https://docs.python.org/3/library/configparser.html
7. **ydata-profiling** is an open-source Python package for advanced exploratory data analysis that enables us to generate the profile reports in fast and simple manner.



# Installing the necessary libraries

In [1]:
#installing faker library
!pip install faker

In [2]:
#installing y_data profiling
!pip install ydata_profiling

# Importing the required libraries

In [4]:
import psycopg2
import pandas as pd
from random import randint
from sql_queries import drop_table_queries,create_table_queries
from faker import Faker
import configparser
from ydata_profiling import ProfileReport

# Establishing a database connection

In [5]:
#configparser module created below is responsible for parsing a list of configuration files, and managing the parsed database.
config = configparser.ConfigParser()

config.read('aws.cfg')

#config.get('section','option') function
host = config.get("LONDON_REGION","host")
user = config.get("LONDON_REGION","user")
dbname = config.get("LONDON_REGION","dbname")
password = config.get("LONDON_REGION","password")

**Here we have used configparser class to get my credentials from  the .cfg file.**
* **The read function is used to read the required .cfg file**
* **get function gives the required parameter in the section mentioned in the function.**
* **For example,in the above function, we are trying to get the credential details of database created in the london region.**

# Create database

In [6]:
def create_database():

    try:
        conn = psycopg2.connect("host=" + host + " user=" + user + " password=" + password)
    except psycopg2.Error as e:
        print(e)

    conn.set_session(autocommit=True)
    curr = conn.cursor()

    curr.execute("drop database if exists hr")
    curr.execute("create database hr")

    curr.close()
    conn.close()

    conn_hr = psycopg2.connect("host=" + host + " user=" + user + " password=" + password + " dbname=" + dbname)
    cur_hr = conn_hr.cursor()

    return conn_hr, cur_hr

Here,
* **The function create_database() is defined to create a PostgreSQL database named "hr" within an RDS instance of AWS.**

* **Inside the function,we establish a connection to the PostgreSQL server using the provided host, user, and password variables.**

* **If an error occurs while connecting to the PostgreSQL server, the psycopg2.Error exception is caught and printed.**

* **The conn.set_session(autocommit=True) line sets the connection to autocommit mode, which means each SQL command will be executed and committed as a separate transaction.**

* **A cursor (curr) is created using the connection. Cursors allow us to execute SQL commands in a database session.**

* **Then we drop the "hr" database if it already exists and create a new "hr" database using the curr.execute() method.**

* **After creating the database, the curr cursor is closed along with the initial connection conn.**

* **A new connection (conn_hr) is established specifically to the "hr" database.**

* **A cursor (cur_hr) is created for this new connection.**

* **The function returns both the conn_hr connection and the cur_hr cursor.**

# Drop the existing tables

In [7]:
#To drop the tables in the database if exists using drop_table_queries module
def drop_tables(conn,cur):
    """
    --Drop all the tables in the database
    """
    for query in drop_table_queries:
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e:
            print("Error : Dropping table failed for" + query)
            print(e)

* **We can also use the database that already exists in the RDS instance. But we need to drop all the tables in the database**
* **Above function is defined for dropping the tables that already exist in the database**
* **Using different .py files as modules in Python allows us to organize our code into separate logical units, which makes our codebase more modular, maintainable, and reusable.**

# Create new tables

In [8]:
#To create required tables using create table queries module
def create_tables(conn, cur):
    """
    --Create the tables using create_table_queries
    """
    for query in create_table_queries:
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e:
            print("Error : Create table failed for " + query)
            print(e)

**Now we define a function to create two tables, "employee" and "city"**

In [9]:
#excecuting all the user defined function to create database, drop old tables and create new tables
conn,cur = create_database()
drop_tables(conn,cur)
create_tables(conn,cur)

# **Using faker to insert the values into the tables created**

In [10]:
#Creating a class Faker to generate fake data
fake = Faker()
fake.random.seed(11)
#defining fake_data_India which generate fake data for India
fake_data_India= Faker("en_IN")

* **After the tables are created, we use the faker library to insert the fake values in the tables**
*  **First, we create class named Faker to generate fake data using the Faker library.**
* **Then the object "fake_data_India" is created to generate fake data which is specifically tailored for India.**
* **The seed function is defined to generate same values everytime we run the function.**

In [11]:
#Using fake_data_India to insert the values into the City table
for i in range (10):
    CityName = fake_data_India.city_name()
    State = fake_data_India.state()
    Pincode = fake_data_India.postcode()
    try:
        cur.execute("INSERT INTO City(CityName,State,Pincode) VALUES (%s, %s, %s)",(CityName,State,Pincode))
    except psycopg2.Error as e:
        print(e)

* **In the "create_table_queries" command, we have defined the column name for city and employee taables. By refering to that file, we generate fake details for existing columns in both the tables**
* **Then we insert the values into the tables**
* **Here, the %s sign is placeholder which inserts the result from the for loop into the table.**

In [12]:
#To look at all the recors in the City table
try:
    cur.execute("select * from City")
except psycopg2.Error as e:
    print(e)

#Fetch all returns rows of a query result, returning them as a list of tuples.
row = cur.fetchall()
row

[(1001, 'Bidhannagar', 'West Bengal', '789328'),
 (1002, 'Fatehpur', 'Rajasthan', '174218'),
 (1003, 'Kalyan-Dombivli', 'Punjab', '792908'),
 (1004, 'Rajkot', 'Arunachal Pradesh', '339075'),
 (1005, 'Gandhidham', 'Odisha', '834701'),
 (1006, 'Berhampore', 'Rajasthan', '681453'),
 (1007, 'Anand', 'Jharkhand', '191614'),
 (1008, 'Kharagpur', 'Assam', '033076'),
 (1009, 'Sikar', 'Maharashtra', '934514'),
 (1010, 'Thoothukudi', 'Andhra Pradesh', '123100')]

**Here we display all the records inserted into city table as list of tuples
(Where each tuple cosists of a row).**

In [13]:
#Creating the dataframe of the list we got from the above query
city_df = pd.DataFrame(row, columns =['CItyID', 'CityName','State','Pincode'])
city_df

,CItyID,CityName,State,Pincode
0,1001,Bidhannagar,West Bengal,789328
1,1002,Fatehpur,Rajasthan,174218
2,1003,Kalyan-Dombivli,Punjab,792908
3,1004,Rajkot,Arunachal Pradesh,339075
4,1005,Gandhidham,Odisha,834701
5,1006,Berhampore,Rajasthan,681453
6,1007,Anand,Jharkhand,191614
7,1008,Kharagpur,Assam,033076
8,1009,Sikar,Maharashtra,934514
9,1010,Thoothukudi,Andhra Pradesh,123100


**We have created city dataframe from the list using pd.DataFrame function**

In [14]:
#Using fake_data_India to insert the values into the employee table
for i in range (30):
    employeefirstname = fake_data_India.first_name()
    employeelastname=fake_data_India.last_name()
    employeecityID = randint(1001,1010)
    try:
        cur.execute("INSERT INTO employee(employeefirstname,employeelastname,employeecityID) VALUES (%s, %s, %s)",(employeefirstname,employeelastname,employeecityID))
    except psycopg2.Error as e:
        print(e)

In [15]:
try:
    cur.execute("select * from employee")
except psycopg2.Error as e:
    print(e)

row1 = cur.fetchall()
#List of employee information tuples stored as list
row1

[(9001, 'Anahita', 'Sur', 1009),
 (9002, 'Aradhya', 'Chakrabarti', 1008),
 (9003, 'Pihu', 'Mammen', 1007),
 (9004, 'Kanav', 'Kapur', 1009),
 (9005, 'Diya', 'Chandran', 1005),
 (9006, 'Sana', 'Sodhi', 1009),
 (9007, 'Hansh', 'Kadakia', 1008),
 (9008, 'Nayantara', 'Handa', 1005),
 (9009, 'Farhan', 'Hora', 1004),
 (9010, 'Aaina', 'Cherian', 1003),
 (9011, 'Aarav', 'Deshmukh', 1002),
 (9012, 'Zaina', 'Rajagopalan', 1009),
 (9013, 'Romil', 'Wali', 1008),
 (9014, 'Akarsh', 'Cheema', 1010),
 (9015, 'Kabir', 'Hegde', 1010),
 (9016, 'Lavanya', 'Rege', 1004),
 (9017, 'Kashvi', 'Bedi', 1005),
 (9018, 'Arnav', 'Bora', 1006),
 (9019, 'Lakshit', 'Kant', 1002),
 (9020, 'Pranay', 'Agrawal', 1008),
 (9021, 'Trisha', 'Ramaswamy', 1001),
 (9022, 'Shalv', 'Thakur', 1001),
 (9023, 'Ritvik', 'Hans', 1008),
 (9024, 'Devansh', 'Barad', 1003),
 (9025, 'Divij', 'Chawla', 1006),
 (9026, 'Keya', 'Ravi', 1001),
 (9027, 'Nirvaan', 'Amble', 1007),
 (9028, 'Kismat', 'Gour', 1008),
 (9029, 'Umang', 'Rastogi', 1004),
 

In [16]:
#creating the data frame of above list as employee df
employee_df = pd.DataFrame(row1, columns =['employeeid','employeefirstname','employeelastname','employeecityID'])
employee_df

,employeeid,employeefirstname,employeelastname,employeecityID
0,9001,Anahita,Sur,1009
1,9002,Aradhya,Chakrabarti,1008
2,9003,Pihu,Mammen,1007
3,9004,Kanav,Kapur,1009
4,9005,Diya,Chandran,1005
5,9006,Sana,Sodhi,1009
6,9007,Hansh,Kadakia,1008
7,9008,Nayantara,Handa,1005
8,9009,Farhan,Hora,1004
9,9010,Aaina,Cherian,1003


**In the same way, we insert values into employee table and create employee dataframe**

# Performing Exploratory data analysis using y_data profiling

In [17]:
city_profile = ProfileReport(city_df)
city_profile.to_file(output_file="city_output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
employee_profile = ProfileReport(employee_df)
employee_profile.to_file(output_file = "employee_output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
city_profile

In [20]:
employee_profile